In [ ]:
import yaml

import numpy as np
import matplotlib.pyplot as plt

from qubic.lib.MapMaking.Qatmosphere_2d import AtmosphereMaps

In [ ]:
# Atmosphere parameters
atm_size = 1e4  # Atmospheric size (meters)
npoints = 1000  # Number of points in each direction

Lx = atm_size  # Atmosphere size in x-direction (meters)
Ly = atm_size  # Atmosphere size in y-direction (meters)
Nx = npoints  # Grid points in x-direction
Ny = npoints  # Grid points in y-direction

# Turbulence parameters
r0 = 1000  # Maximal turbulence size (meters) - Typical values for Atacama desert are on the order of several hundred meters from Morris(2020) and Errard (2015)
k_r0 = 2 * np.pi / r0  # Wavenumber corresponding to r0

# How to compute water vapor density variance from PWV

## Definitions

**Precipitable Water Vapor (PWV)** is defined as the total water vapor in the atmosphere expressed in millimeters (mm). It represents the equivalent depth of liquid water if all the water vapor were condensed.

**Water Vapor Density** ($\rho_{wv}$) is the mass of water vapor per unit volume, typically expressed in $\text{g/m}^3$.

The relationship between PWV and water vapor density is given by:
$
\text{PWV} = \int_0^H \rho_{wv}(z)\,dz,
$
where:
- $H$ is the height of the atmospheric column,
- $\rho_{wv}(z)$ is the water vapor density as a function of altitude $z$.

## Assumptions

1. **Uniform Water Vapor Density:**  
   It is assumed that
   $
   \rho_{wv}(z) \approx \rho_{wv}
   $
   (i.e., the density is constant throughout the column).

2. **Atmospheric Height:**  
   The effective height of the water vapor layer is approximated as:
   $
   H \approx 2000\,\text{m}.
   $

## Relation Between PWV and Density

If $\rho_{wv}$ is uniform, then:
$
\text{PWV} = \rho_{wv} \cdot H.
$
Thus, the water vapor density can be expressed as:
$
\rho_{wv} = \frac{\text{PWV}}{H}.
$

For example, for $\text{PWV} = 1\,\text{mm} = 1\times10^{-3}\,\text{m}$ and $H = 2000\,\text{m}$:
$
\rho_{wv} = \frac{1\times10^{-3}}{2000} = 5\times10^{-7}\,\text{m}^{-1}.
$
Converting this to $\text{g/m}^3$ (by multiplying by $10^6$):
$
\rho_{wv} = 0.5\,\text{g/m}^3.
$

## Variance of Water Vapor Density

If $\sigma^2_{\text{PWV}}$ is the variance of PWV, then the variance of the water vapor density is given by:
$
\sigma^2_{\rho_{wv}} = \frac{\sigma^2_{\text{PWV}}}{H^2}.
$

Assuming:
$
\sigma^2_{\text{PWV}} = (1\,\text{mm})^2 = (1\times10^{-3}\,\text{m})^2 = 1\times10^{-6}\,\text{m}^2,
$
with $H = 2000\,\text{m}$, we have:
$
\sigma^2_{\rho_{wv}} = \frac{1\times10^{-6}}{(2000)^2} = \frac{1\times10^{-6}}{4\times10^6} = 2.5\times10^{-13}\,\text{m}^{-2}.
$

Multiplying by $(10^6)^2$ to convert to $\text{g}^2/\text{m}^6$:
$
\sigma^2_{\rho_{wv}} = 0.25\,\text{g}^2/\text{m}^6.
$

## Summary

- **Mean Water Vapor Density:**
  $
  \rho_{wv} = 0.5\,\text{g/m}^3.
  $

- **Variance of Water Vapor Density:**
  $
  \sigma^2_{\rho_{wv}} = 0.25\,\text{g}^2/\text{m}^6.
  $


In [ ]:
sigma_wv = 0.25  # Variance of the water vapor density (g²/m⁶)

In [ ]:
dx = Lx / Nx
dy = Ly / Ny

# Wavenumbers (radians per meter)
kx = 2 * np.pi * np.fft.fftfreq(Nx, dx)
ky = 2 * np.pi * np.fft.fftfreq(Ny, dy)

kx_2d, ky_2d = np.meshgrid(kx, ky, indexing="ij")
k_magnitude = np.sqrt(kx_2d**2 + ky_2d**2)

In [ ]:
def kolmogorov_spectrum(k_magnitude, k_r0, sigma_wv):
    # Compute normalization constant C
    sum_kk = np.sum((k_r0**2 + k_magnitude**2) ** (-8 / 6))
    C = (sigma_wv * Lx * Ly) / sum_kk

    # Initialize power spectrum
    P = np.zeros_like(k_magnitude)
    P = C * (k_r0**2 + k_magnitude**2) ** (-8 / 6)

    return P


P = kolmogorov_spectrum(k_magnitude, k_r0, sigma_wv)

In [ ]:
# Plot the power spectrum in one dimension
plt.figure(figsize=(8, 6))
plt.plot(
    k_magnitude[0],
    kolmogorov_spectrum(k_magnitude[0], k_r0, sigma_wv),
    label="1D Power Spectrum",
)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Wavenumber")
plt.ylabel("Power Spectrum")
plt.title("Kolmogorov Power Spectrum")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Generate random Fourier coefficients with Hermitian symmetry
np.random.seed(0)  # For reproducibility
noise_real = np.random.normal(0, 1, (Nx, Ny))
noise_imag = np.random.normal(0, 1, (Nx, Ny))
noise = noise_real + 1j * noise_imag

# Scale by sqrt(P) and handle symmetry
F = noise * np.sqrt(P)
F_symmetric = np.fft.fft2(np.fft.ifft2(F).real)  # Ensures real field

In [ ]:
# Perform inverse FFT
rho = np.fft.ifft2(F_symmetric).real

# Normalize to ensure correct variance
current_var = np.var(rho)
rho *= np.sqrt(sigma_wv / current_var)

In [ ]:
plt.imshow(rho, extent=[0, Lx / 1e3, 0, Ly / 1e3], cmap="jet")
plt.colorbar(label="Water Vapor Density (g/m^3)")
plt.xlabel("x (km)")
plt.ylabel("y (km)")
plt.title("2D Atmospheric Water Vapor Density Fluctuations")
plt.show()

# Build Atmosphere class

In [ ]:
params = yaml.safe_load(open("params.yml", "r"))
atm = AtmosphereMaps(params)

# Build atm map

In [ ]:
temp = atm.get_temp_maps(rho)

In [ ]:
plt.imshow(temp[0], extent=[0, Lx / 1e3, 0, Ly / 1e3], cmap="jet")
plt.colorbar(label="Temperature (µK)")
plt.xlabel("x (km)")
plt.ylabel("y (km)")
plt.title("2D Atmospheric Temperature Fluctuations")
plt.show()